In [4]:
#load aiida environment and connect to database
from aiida import load_profile
from aiida.orm import computers
profile = load_profile()
print('profile name : ',profile.name)
print('profile database host name : ',profile.database_hostname)
print('default user : ',profile.default_user)

profile name :  default
profile database host name :  localhost
default user :  r.mozumder@fz-juelich.de


In [5]:
# load classes and functions
from aiida.orm import CifData, Code, Dict, load_node, StructureData 
from aiida.orm import (Group, load_group, load_node, load_code, groups,
                      WorkChainNode, QueryBuilder)
from aiida.engine import submit
from aiida.common.exceptions import NotExistent
from aiida_kkr.workflows import kkr_imp_sub_wc, kkr_imp_dos, kkr_imp_wc, kkr_startpot_wc
import numpy as np
from aiida_kkr.calculations import KkrimpCalculation, VoronoiCalculation
import matplotlib.pyplot as plt
from aiida_kkr.tools.common_workfunctions import get_username

In [23]:
# Structure data from voronoi calc or kkr_startpot_wc
def Struc_from_voro(voro_node, host_structure=True, imp_structure=False):
    # Packages and module
    from aiida.common.exceptions import InputValidationError
    from aiida_kkr.calculations import VoronoiCalculation
    from aiida_kkr.workflows import kkr_startpot_wc 
    from aiida.orm import StructureData, Node
    """
    Structure from the voronoi calcjob or voro_startpot_wc
    voro_node: (voro_calc)
    """
    # To check voro_node in node or ID
    if not isinstance(voro_node, Node):
        try:
            voro_node = load_node(voro_node)
        except IndentationError:
            print('No such pk or uuid: {} is present in the database.'.format(voro_node))
            
    host_struc, imp_host_struc = None, None
    
    # For voronoi calcjob
    if (voro_node.process_class==VoronoiCalculation) :
        ## Degug here
        print('This Voronoicalculation')
        struc = voro_node.inputs.structure
        # Check for is it from ancector wc or created there
        # For example the struc from sub kkr_startpot_wc in kkr_imp_wc is created there so 
        # no inconing_node is available 
        
        if struc.get_incoming().first() == []:
            imp_host_struc = struc
    # For kkr_startpot_wc  
        
    

In [ ]:
####### NOTE: If this csucessful then add it into the 'tools_development' ipynb
## In this part the nodes from a group having with different class and kind of nodes, 
## can be collected to store in new group only considering the kkr_imp_wc node

group_wasmer = load_group(83)
# node_label
debug= False
wasmer_grp_list = list(group_wasmer.nodes)
group_label = 'imp_embedded_Au_by_wasmer'
try:
    new_grp = load_group(group_label)
    new_grp_list = list(new_grp.nodes)
    new_grp_list =  [node.pk for node in new_grp_list]
    print('Group as named %s is already stored, so no need to create the group'%group_label)
except:
    new_grp = Group(label=group_label)
    new_grp.store()
    print(' A new node created as named %s' %node_label)

i=0
imp_wc_no = 0
for index in range(len(wasmer_grp_list)):
    node = wasmer_grp_list[index]
    if node.node_type.split('.')[-2]=='WorkChainNode':
        if (u'kkr_imp_wc' == node.process_label):
            imp_wc_no += 1
            if node.pk in new_grp_list[:]:
                print('node {} is already exist'.format(node.pk))
            else:
                new_grp.add_nodes(node)
#             if debug:
#                 print(node.process_label)
#                 print('pk: %7d'%node.pk)
#                 print(wasmer_grp_list[i])
#                 print(node.label)
                
                        
        


In [118]:
## Trying to develop to delete the node from database
## and at the same time from the remote folder

## Add the following possible services
# 1. Give the print option for how many decendant node will be 
#       and take the permission. Add option wether need to take 
#         permission or not.
# 2. Also print how many calcjob node will be deleted under permission 
# 3. Print'remote directory' to check that all the data from the
#     remote dir are deleted or not.
## Technique
# 1. Use the QuaryDB()
# Use the cleandir



#Section-1: delete node from the database as well as remote work directory
### A function to delete the data of calculation output of calcjob list (pks) from the remote dir.

# later add it with the del_node  function
# This is successfully done

# please note that it is tested for one calc list
def delete_remote_workdir(pks, verbosity=0, dry_run= True):
    from aiida.common import exceptions
    from aiida.orm import load_node
    from aiida.orm import computers
    import sys
    # TODO : add the verbosity as discused here 
    # https://aiida.readthedocs.io/projects/aiida-core/en/v1.5.0/_modules/aiida/manage/database/delete/nodes.html

    """
    :param pks: calc node list
    
    """
    removed_path_list = [] # The part of the path will be deleted
    remote_path_list = []  # The original path
    updated_path_list = [] # After removing the part of the path
    loadable_list = [] # To load the node and save it loadable_list
    loaded_node_list = []
    # To check the loadable calcjob list
    for pk in pks:
        try:
            loaded_node = load_node(pk)
        except exceptions.NotExistent:
            print('This is calcjob node'.format(pk))
            loaded_node = pk
            loaded_node_list.append(loaded_node)
            #             sys.exit()
        else:
            loaded_node_list.append(loaded_node)
    # Computer data
    
   
    for node in loaded_node_list:
        load_pk = node
        # computer data
        computer = load_pk.computer
        computer_name = computer.label
        print(computer_name)
        
        remote_path = load_pk.get_remote_workdir()
        remote_path_list.append(remote_path)
        
        delete_folder = remote_path.split('/')[-1]
        removed_path_list.append(delete_folder)

        new_remote_path = remote_path.replace(remote_path.split('/')[-1], '')
        updated_path_list.append(new_remote_path)
        
        
        
    if dry_run or verbosity==3:

        for i, paths in enumerate(zip(remote_path_list, updated_path_list)):
            print('Before the delation the original path list : {}\n'.format(paths[0]))
            print('After deletion the modefied or updated path : {}'.format(paths[1]))
    val = input("Are you agree to clean the remote workdir (y/n) : ")
    
    if str(val)=='y' or str(val)=='Y':
        for remote_path in remote_path_list:
            if not dry_run:
                    # Open the connection to the remote folder/dir via transport
                    computer_transport = computer.get_transport()
                    is_transport_open = computer_transport.is_open
                    if not is_transport_open:
                        computer_transport.open()
                    try:
                        computer_transport.rmtree(remote_path)
                    except IOError as ex:
                        print(ex)
    else:
        print('Nothing to clean from the remote workdir!')
# section-5:delete node from the database as well as remote work directory
## It returns all the calcjob from a WC node
def find_calcJob(pk_or_node, debug=True):
    
    calcjob_node_list=[]
    wc_node_list = []
    try:
        if isinstance( pk_or_node, int):
            if debug:
                print('This is pk')
            node = load_node(pk_or_node)
        else:
            if debug:
                print('This is node.')
            node= pk_or_node
    except:
        print('{} is nither node ID nor aiida_node. '.format(pk_or_node))
        
    ## Use the get_calcjob_wc to get descendent calcjob list and  wc list
    calc_list, wc_list = get_calcjob_wc(node)
    calcjob_node_list += calc_list
    
    while len(wc_list)!=0:
        new_wc_list = []

        for i in wc_list[:]:
            calc_list, wc_list = get_calcjob_wc(i)
            new_wc_list += wc_list
            calcjob_node_list += calc_list
            
        wc_list = new_wc_list

    return calcjob_node_list

## This function returns calcjob_list and wc_list from a wc or calcjob node   
def get_calcjob_wc(node):
    """
    :param: node
    :return: workchain node list and calcjob node list
    """ 
    from aiida.orm import CalcJobNode, WorkChainNode
    wc = []
    calc_job = []
    
    if node.node_type == 'process.workflow.workchain.WorkChainNode.':
        
    # here all outgoing worchain node
        out_going_wc = node.get_outgoing(node_class=WorkChainNode).all()
        wc = [i.node for i in out_going_wc[:]]
        
    # here all outgoing calcjob node
        out_going_calc = node.get_outgoing(node_class=CalcJobNode).all()
        calc_job = [i.node for i in out_going_calc[:]]
                    
    elif node.node_type == 'process.calculation.calcjob.CalcJobNode.':
        calc_job.append(node)
    
    return calc_job, wc

# This is the final del_node_function. Using this function for any specific wc node the node from the 
# Db as well as the calcjob data from the remote workdir can be deleted.
def del_node(node_pks, dry_run=True, verbosity=3, debug=True):
    """
    1. This function will delete the node data from the database and also from the remote_dir
    
    :params node_pks: (list) list of workchain to delete from database as well as from remote workdir
    :param verbosity: 0 prints nothing.  This is for workdir and wc
                      1 prints just sums and total.   This is for workdir but not for wc
                      2 prints indivisual nodes.  This is for workdir and wc
    :param dry_run: Do not delete anything just show the status as in the verbosity given
    """
    from aiida.orm import load_node
    from aiida.manage.database.delete.nodes import delete_nodes
    
        
    calcjobs_list = []
    pks_given = []
    for i in node_pks:    
        try:
            if isinstance( i, int):
                if debug:
                    print('This might be pk or uiid')
                node = load_node(i)
            else:
                if debug:
                    print('This might be a node.')
                node= i
        except:
            print('{} is nither node ID nor aiida_node. '.format(i))
        
        pks_given.append(node.pk)
        
        calcjobs = find_calcJob(node, debug)
        calcjobs_list += calcjobs
        print('calcjob list : ', calcjobs_list,)
    delete_remote_workdir(calcjobs_list, verbosity=verbosity, dry_run=dry_run)
    delete_nodes(pks_given, verbosity=verbosity, dry_run=dry_run,force=False)
    

In [119]:
node = load_node(24726)
del_node([node], verbosity=3,dry_run=False)

This might be a node.
This is node.
calcjob list :  [<CalcJobNode: uuid: ed304c61-9cf5-4fac-a1ea-25ffb30c862d (pk: 24769) (aiida.calculations:kkr.kkrimp)>, <CalcJobNode: uuid: caf85a4d-ac7c-4774-8e3c-ced62cf72ca2 (pk: 24737) (aiida.calculations:kkr.kkr)>, <CalcJobNode: uuid: 12498ecc-db24-4209-a0bb-1bda045a9604 (pk: 24752) (aiida.calculations:kkr.kkrimp)>, <CalcJobNode: uuid: 08bf1aba-fa77-4e70-af6e-85ffdf14eaf1 (pk: 24759) (aiida.calculations:kkr.kkrimp)>]
This is calcjob node
This is calcjob node
This is calcjob node
This is calcjob node
claix18_init
claix18_init
claix18_init
claix18_init
Before the delation the original path list : /rwthfs/rz/cluster/work/jara0191/ck142666/aiida/computers/claix18_init/ed/30/4c61-9cf5-4fac-a1ea-25ffb30c862d

After deletion the modefied or updated path : /rwthfs/rz/cluster/work/jara0191/ck142666/aiida/computers/claix18_init/ed/30/
Before the delation the original path list : /rwthfs/rz/cluster/work/jara0191/ck142666/aiida/computers/claix18_init/ca/f8/

Are you agree to clean the remote workdir (y/n) :  n


Nothing to clean from the remote workdir!
I will delete 37 nodes
The nodes I will delete:
   53d9b69b-9272-40af-b9cc-20937f6858dc 24726 WorkChainNode pk16195:16195
   a520f58e-423d-49bf-867f-d4a3bd469ef8 24727 CalcFunctionNode create_combined_imp_info_cf
   a880e382-b261-49a6-8bf4-df93021d4bf1 24728 Dict 
   e4df6f3f-e767-468b-a0ec-816e4273283d 24729 Dict 
   abd8e4e8-62e3-45fe-8b7c-11ca50749175 24732 WorkChainNode GF writeout combined imps
   0d599420-c985-4034-9444-b3fc621d0655 24734 CalcFunctionNode update_params_wf
   557b6a7f-2df0-4889-a50b-d28d7bdb5ffc 24735 Dict updated KKR parameters
   caf85a4d-ac7c-4774-8e3c-ced62cf72ca2 24737 CalcJobNode KKRFLEX calc.
   e710b6f3-2cb5-4a7a-aa25-2ae96af15e0a 24738 RemoteData 
   3f3cdb45-9127-424a-83c1-68b07b292e61 24739 FolderData 
   69728675-9650-48db-8b9e-e40590fec5d1 24740 Dict 
   ac198223-2efc-4f1d-8dfb-5e3753d99176 24742 CalcFunctionNode create_out_dict_node
   26bed398-a0ce-4eae-8fc4-e87333df0a92 24743 Dict kkr_flex_wc_results
   eb3

 n


Exiting without deleting


In [38]:
find_calcJob(24624)

[<CalcJobNode: uuid: 4f96b1e1-7bf9-442e-8a50-5147b733e794 (pk: 24667) (aiida.calculations:kkr.kkrimp)>,
 <CalcJobNode: uuid: b2b07931-8919-4216-ab1d-179be5f02979 (pk: 24635) (aiida.calculations:kkr.kkr)>,
 <CalcJobNode: uuid: 8cbbabd5-8288-469b-a045-9569e8a14876 (pk: 24650) (aiida.calculations:kkr.kkrimp)>,
 <CalcJobNode: uuid: 79ae3a10-9c5d-4b40-b44b-37bb294c9f81 (pk: 24657) (aiida.calculations:kkr.kkrimp)>]

In [40]:
# section: 1.1 delete node from the database as well as remote work directory
delete_remote_workdir([24650,24657],verbosity=3, dry_run=False)

claix18_init
claix18_init


In [ ]:
20046 = ck142666/aiida/computers/claix18_init/33/72/09bd-3958-401f-83f2-1fdbe3411e74
20024 = ck142666/aiida/computers/claix18_init/17/10/feef-b826-453e-a5d8-58b8d1dd08ff
20010 = ck142666/aiida/computers/claix18_init/5e/f7/5c5f-0355-446d-94c5-4466340e8817
19995 = ck142666/aiida/computers/claix18_init/b5/e3/6d58-837a-454c-8c9c-d380454a4409
19972 = ck142666/aiida/computers/claix18_init/f2/33/0597-face-495e-be74-b321c7ee6689
19967 = ck142666/aiida/computers/claix18_init/5f/f0/b136-ccfe-4f81-b246-65cbfcbf42c7

In [33]:
#section-2: delete node from the database as well as remote work directory
## This code  will help to workchain as explained in the filter
qb = QueryBuilder()
node_list = list(qb.append([WorkChainNode],
                    filters={
                        'and':[
                            {'attributes.process_label':'combine_imps_wc'},
                            {'or':[
                                {'attributes.process_status': 'excepted'},
                                {'attributes.exit_status':{'!in':[0]}},
                                {'attributes.process_status': 'killed'},
                                
                                 ]
                            }
                        ]
                    }
                   
                   
                   ).all())

# node_list

In [35]:
## section-3 : delete node from the database as well as remote work directory
## To check any specific pk in the pk_list exist or not
def check_pk_exist(pk_list, pk):
    for i in pk:
        for j in pk_list:
            if(i==j):
                print('pk-{} is exist in pk_list'.format(i))
                continue

len(node_list)

pk_list = [i[0].pk for i in node_list]
pk_list.sort()
print(check_pk_exist(pk_list,[22232]))

None


In [55]:
# Section-4 :delete node from the database as well as remote work directory
from aiida.manage.database.delete.nodes import delete_nodes
ll = delete_nodes([24675], verbosity=3, dry_run=True,force=False)

I would delete 36 nodes
The nodes I would delete:
   9825dd04-078f-4883-8467-649ba8498a03 24675 WorkChainNode pk16195:16195
   d7ed3298-cdf0-4901-9ff4-b2a1bde63540 24676 CalcFunctionNode create_combined_imp_info_cf
   84a56bc1-cbc2-483f-9956-201993b9c349 24677 Dict 
   57583625-803a-41ea-9200-74dd344b8404 24678 Dict 
   b760a2d4-e248-480d-b88a-3295ece98213 24681 WorkChainNode GF writeout combined imps
   aaf8f48a-24b6-4e9d-b835-3e30ced53b46 24683 CalcFunctionNode update_params_wf
   d16738f8-5845-49b6-ab85-66761a8c7a6a 24684 Dict updated KKR parameters
   174224c2-65cc-4efd-8b2f-a6da26e834de 24686 CalcJobNode KKRFLEX calc.
   91421b08-88b0-4c94-97ff-09f7a77fcabc 24687 RemoteData 
   802be422-058f-43e5-a8d9-d918e3fd1aa4 24688 FolderData 
   a494fdfb-ea17-4071-bb19-c5f34d0ef777 24689 Dict 
   59e63cfe-bc23-43fd-898b-f06efcaa50e3 24691 CalcFunctionNode create_out_dict_node
   9c4bf5f1-9f27-46a2-9195-d912d06d509e 24692 Dict kkr_flex_wc_results
   5af11ebe-cb78-4eb1-8695-974f321ad7b6 24694 

In [54]:
ll